In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
from langchain.tools import tool
from typing import Dict, Any
from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase

tavily_client = TavilyClient()

db = SQLDatabase.from_uri("sqlite:///C:/Users/SHUBHAM SARKAR/Desktop/Agentic/lca-lc-foundations/notebooks\module-3/resources/Chinook.db")


@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [16]:
from dataclasses import dataclass

@dataclass
class UserRole:
    user_role: str = "external"

In [17]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Dynamically call tools based on the runtime context"""

    user_role = request.runtime.context.user_role
    
    if user_role == "internal":
        pass # internal users get access to all tools
    else:
        tools = [web_search] # external users only get access to web search
        request = request.override(tools=tools) 

    return handler(request)

In [18]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=[web_search, sql_query],
    middleware=[dynamic_tool_call],
    context_schema=UserRole
)

In [ ]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "internal"}
)

print(response["messages"][-1].content)

275

Note: The table is named Artist (capital A). The initial query used 'artists' (lowercase), which doesn’t exist in this schema. If you’d like, I can list the first few artist names or run more stats (e.g., number of albums per artist).


In [20]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "external"}
)

print(response["messages"][-1].content)

I don’t have access to your database, so I can’t see the exact count. If you tell me the database type and the relevant table/collection, I can give you the exact query. Here are common options you can run:

- SQL (e.g., a table named artists)
  - Total artists: SELECT COUNT(*) AS total_artists FROM artists;
  - If you want distinct artists from artworks (artist_id in artworks): SELECT COUNT(DISTINCT artist_id) AS total_artists FROM artworks;
  - Excluding inactive/soft-deleted: add a where clause, e.g., WHERE active = true or WHERE deleted_at IS NULL.

- PostgreSQL/MySQL/MSSQL (similar syntax)
  - Total from artists: SELECT COUNT(*) FROM artists;
  - Distinct artists in artworks: SELECT COUNT(DISTINCT artist_id) FROM artworks;

- NoSQL (MongoDB)
  - From artists collection: db.artists.countDocuments({});
  - Distinct artists used in artworks: db.artworks.distinct("artist_id").length;

- If you use an API
  - GET /artists with a count parameter or a dedicated GET /artists/count endpoin